# 6.04
## 조건부확률 P(A, B)/P(B) = P(A|B)
### 결합확률을 알면 개별확률, 조건부 확률 둘다 찾을 수 있다.
## 결합확률을 어떻게 구할거냐면 chain rule 적용해서 P(A|B, C,.. Z)P(B|C ... Z) ...
## NAIVE BAYES
## Bayes Theorm
## mle는 경험만 가지고 측정한다. 사전확률을 추가해서 map로 

In [1]:
trainingData =[
    (1, "Chinese Beijing Chinese", True),
    (2, "Chinese Chinese Shanhai", True),
    (3, "Chinese Macao", True),
    (4, "Tokyo Japan Chinese", False)
]

testingData = (5, " Chinese Tokyo Japan")

In [2]:
V = list(set(([term for _ in trainingData
        for term in _[1].lower().split()])))
V

['shanhai', 'beijing', 'tokyo', 'macao', 'chinese', 'japan']

In [3]:
N = len(trainingData)
N

4

In [4]:
trueData = [_ for _ in trainingData if _[2]]
falseData = [_ for _ in trainingData if not _[2]]

In [5]:
trueData

[(1, 'Chinese Beijing Chinese', True),
 (2, 'Chinese Chinese Shanhai', True),
 (3, 'Chinese Macao', True)]

In [6]:
from collections import defaultdict

prior = list()

        
Tct = defaultdict(int)
for data in trueData:  
    Nc = len(trueData)
    PriorC = Nc/N
    for term in data[1].lower().split():
        Tct[term] += 1

_Tct = defaultdict(int)
for data in falseData:  
    _Nc = len(falseData)
    _PriorC = _Nc/N
    for term in data[1].lower().split():
        _Tct[term] += 1

In [7]:
Tct

defaultdict(int, {'chinese': 5, 'beijing': 1, 'shanhai': 1, 'macao': 1})

In [8]:
condProbC = defaultdict(float)
_condProbC = defaultdict(float)

countSum = sum(Tct.values())
_countSum = sum(_Tct.values())

for term, freq in Tct.items():
    condProbC[term] = (freq+1)/(countSum + len(V))

for term, freq in _Tct.items():
    _condProbC[term] = (freq+1)/(_countSum + len(V))

In [9]:
countSum, _countSum

(8, 3)

In [10]:
condProbC, _condProbC

(defaultdict(float,
             {'chinese': 0.42857142857142855,
              'beijing': 0.14285714285714285,
              'shanhai': 0.14285714285714285,
              'macao': 0.14285714285714285}),
 defaultdict(float,
             {'tokyo': 0.2222222222222222,
              'japan': 0.2222222222222222,
              'chinese': 0.2222222222222222}))

In [11]:
# P(C)multi(P(Tct|C)) ===> log(P(C)) + SUM(P(Tct|C))
from math import log, exp

# Prior Probability
result = log(PriorC)
_result = log(_PriorC)

# joint probability => conditional independence
for term in testingData[1].lower().split():
    # multi -> log sum
    result += log((Tct[term]+1)/(countSum+len(V)))
    _result += log((_Tct[term]+1)/(_countSum+len(V)))

# optimal classfier
if result > _result:
    print("True", result, exp(result))
else:
    print("False", _result, exp(_result))

False -5.898526551448713 0.0027434842249657062


In [12]:
condProbC["chinese"], _condProbC["chinese"], _condProbC["tokyo"] 

(0.42857142857142855, 0.2222222222222222, 0.2222222222222222)

# sklearn 뉴스데이터 예제

In [37]:
from sklearn.datasets import fetch_20newsgroups
newsdata=fetch_20newsgroups(subset='train')
print(newsdata.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


In [14]:
print (len(newsdata.data), len(newsdata.filenames), len(newsdata.target_names), len(newsdata.target))

11314 11314 20 11314


In [18]:
print(newsdata.target_names)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [19]:
print(newsdata.target[0])
print(newsdata.target_names[7])

7
rec.autos


In [20]:
print(newsdata.data[0])

From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----







In [30]:
from sklearn.feature_extraction.text import CountVectorizer

tdmvector = CountVectorizer()
X_train_tdm = tdmvector.fit_transform(newsdata.data)

print(X_train_tdm.shape)

(11314, 130107)


In [31]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
tfidfv = tfidf_transformer.fit_transform(X_train_tdm)
print(X_train_tdm.shape)

(11314, 130107)


In [32]:
X_train_tdm[1]

<1x130107 sparse matrix of type '<class 'numpy.int64'>'
	with 93 stored elements in Compressed Sparse Row format>

In [33]:
from sklearn.naive_bayes import MultinomialNB # 다항분포 나이브 베이즈 모델
mod = MultinomialNB()
mod.fit(tfidfv, newsdata.target)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [34]:
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [35]:
from sklearn.metrics import accuracy_score #정확도 계산을 위한 함수
newsdata_test = fetch_20newsgroups(subset='test', shuffle=True) #테스트 데이터 갖고오기
X_test_tdm = tdmvector.transform(newsdata_test.data) #테스트 데이터를 TDM으로 변환
tfidfv_test = tfidf_transformer.transform(X_test_tdm) #TDM을 TF-IDF 행렬로 변환

predicted = mod.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(newsdata_test.target, predicted)) #예측값과 실제값 비교

정확도: 0.7738980350504514
